<h1 style="color:brown;">  Recurrent neural nets</h1> 

![Looping network](./img/RNN_colah.png)

##### RNNs can produce amazing results <a href ="http://karpathy.github.io/2015/05/21/rnn-effectiveness/">blog</a>

### Lesson plan 
1. Why classic neural nets are not enough?
2. Word embeddings - word2vec
3. Categorical embeddings
4. RNN 
5. Takeaways
6. Hands on word2vec

In [3]:
import numpy as np

### Classic nets vs. RNN's

Classic:
    - Inputs and outputs must be fixed-sized vectors
    - No idea of location or time 

RNNs: 

![](./img/diags.jpeg)

### Word embeddings

#### N-gram

-Which word produces the highest probability to be next given we have seen n specific other words before

In [ ]:
Words: Thank, you, Hello, goodbye

In [ ]:
If we have 4 words and we are looking at 2-gram? 
    Example: no. of times Thank you occurs divided by number of times Thank occurs

We need to calculate the probabilty of 
 - Thank Hello
 - Thank you
 - Thank goodbye
 - Thank Thank

So we needed to do 4 calculations

In [17]:
def how_many_calc_to_do(gram, voc_size):
    '''This function needs to calculate all combos 
    of words'''
    
    return np.prod(np.repeat(voc_size, gram))

In [15]:
how_many_calc_to_do(7, 10000)
# Notice that this is only an approxiamtion and it can be implemented in more efficient ways.

4477988020393345024

![](./img/one_hot_encoding_distance_on_3d.png)

#### Insight I: 
    we can actually just turn each word to a random vector sized 100/200/300, 
    train a classic neural net to predict the next word and update both the weights and the random vectors.
    You can think of it as just another layer of weights multiplying the one hot encoded inputs.
<a href="http://hunterheidenreich.com/blog/intro-to-word-embeddings/">word_embed blog</a>

<a href="https://towardsdatascience.com/introduction-to-word-embedding-and-word2vec-652d0c2060fa">word_embed blog II</a>


![W2V](./img/w2v.png)

##### Objective: maximize the sum of probabilities of each word given its observed window

This idea is very strong in comparison to other options: 

    Bag of words - just count occourences 
    TF-IDF - word is either informative or not but has no relation to other words
    one-hot encoder: for the computer paris-france is the same distance as paris-blabla

- Distance and direction are meaningful! King - man = Queen - woman
- Now the words massive and huge are similar!
- Extends to sentences, paragraphs and documents.

![](./img/see_attached_word_embed.png)

##### We have reduced the dimension of the vocublary by a big factor!
example: from 80,000 to 300

![](https://www.lemay.ai/demo/wordEmbedding/)

##### Insight I.I  The same thing can be applied to any categorical variable. 
##### With enough training data we can learn its continous position in space - state of the art

![categorical_embed](./img/categorical_embedding.png) # image I

![german_states](./img/german_states_mapped_2D.png) 

#### Insight II: 
        well, even if we can include many words (large n-gram), how can we capture context?
        If the text mentioned queen Mary and few pages later is talking about the queen, how will our network 
        know her name is Mary? 

### Idea I: Memory - your current choices are based on previous understanding

Add some cell in the network to keep previous memory and combine with current input to predict next word

![](./img/memory_rnn.png)

#### Problem: calculating the derivative (aka gradient) is problematic, either infinite or zero.

Imagine the memory at time t is the memory at time t-1 times a weight vector:
    $h_t = W*h_{t-1}$
Then:
    $h_t = W^t * h_0$ 
    
  $W > 1$ $h_t --> \infty$

### Solution: LSTM/GRU

<a href="https://colah.github.io/posts/2015-08-Understanding-LSTMs/">LSTM/GRU blog</a>


![](./img/RNNs.png)

![](./img/LSTM_colah.png)

#### Idea II: gates: don't multiply, use addition for memory!

##### Components

    - cell state
    - candidates  

##### Gates
- forget - information to throw (0 means throw all from the cell state)
- input - what values we are going to update
- output - filter which values of the cell we are going to output 

The current cell state is the sum of forgetting and updating with new candidates

### Extension: attention

<a href="https://www.youtube.com/watch?v=SysgYptB198">Intuition</a>

######  - Translate part by part
###### -  Use attention weights - how much attention should you give to each word in the input (update weights to each new word)

![](./img/attention.png)

### Takeaways:
    

##### Word embeddings
- Word/categorical embeddings gives meaning to words in relation to one another
- Word/categorical embeddings are computationally efficient
- Training is done through a classic NN with small window around words

##### RNN
- Old generation RNNs suffered from exploding/vanishing gradients
- New generation RNNs (commonly LSTM or GRU) are using memory gates to mitigate this problem
- RNNs are just multiple copies of a NN connected by the hidden layer
- Training is done again by backpropogation
- Weights are shared accros all network
- RNN's can be used for any sequence. Unlike time series models can include both time and features.
- Are flexible in input and output sizes
- Amazing results in NLP, recommendations and many more.
- Many flavours - BRNN, CRNN...

##### Attention
- Typicall for translations/images
- Weight all the words in one language to decide how much they should influence input to translated language
- components: word weights, BRNN, RNN, context vectors.

### Hands on Word2Vec/word embedding

In [ ]:
!pip install gensim

In [3]:
import gensim
import numpy as np
import json
import string

##### Reading in the data

In [4]:
with open('/Users/OmersGuest/Documents/dc_ds_04_22_19/module_6/JEOPARDY_QUESTIONS1.json') as f:
    data = json.load(f)

In [5]:
len(data)

216930

In [6]:
# Let's look at the first element in our list
data[0]

{'category': 'HISTORY',
 'air_date': '2004-12-31',
 'question': "'For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory'",
 'value': '$200',
 'answer': 'Copernicus',
 'round': 'Jeopardy!',
 'show_number': '4680'}

In [7]:
# Word2Vec requires that our text have the form of a list
# of 'sentences', where each sentence is itself a list of
# words. How can we put our _Jeopardy!_ clues in that shape?

text = []

for clue in data:
    sentence = clue['question'].translate(str.maketrans('', '',
                                                        string.punctuation)).split(' ')
    
    new_sent = []
    for word in sentence:
        new_sent.append(word.lower())
    
    text.append(new_sent)

In [8]:
# Let's check the new structure of our first clue
text[0]

['for',
 'the',
 'last',
 '8',
 'years',
 'of',
 'his',
 'life',
 'galileo',
 'was',
 'under',
 'house',
 'arrest',
 'for',
 'espousing',
 'this',
 'mans',
 'theory']

#####  Constructing the model

In [9]:
# simply a matter of
# instantiating a Word2Vec object.
model = gensim.models.Word2Vec(text, sg=1)
## sg means skip-gram

##### training 

In [10]:
# To train, call 'train()'!
model.train(text, total_examples=model.corpus_count, epochs=model.epochs)

(11336570, 15849970)

In [11]:
# The '.wv' attribute stores the word vectors
model.wv

In [12]:
model.wv['child']

array([-0.2660608 ,  0.74738264,  0.6810505 , -0.47299108,  0.05217691,
       -0.10152421, -0.20832658,  0.08030565,  0.32037446,  0.3153305 ,
       -0.34866828, -0.31779078,  0.22926913,  0.60894746, -0.56666803,
        0.01631304, -0.31678593, -0.20412245,  0.27447847, -0.67274284,
        0.36620042, -0.23380297,  0.13526577,  0.2926762 ,  0.18137805,
       -0.2558905 ,  0.23138794,  0.04789907,  0.21861915,  0.24400485,
       -0.3116787 ,  0.47753632, -0.14129499,  0.03673249,  0.19247477,
        0.26503217, -0.09043661,  0.14087467, -0.3829878 ,  0.29696804,
       -0.05752321,  0.8588681 ,  0.14684854,  0.38990337,  0.09335101,
       -0.40334013,  0.0594354 ,  0.12365463, -0.0052815 ,  0.30034325,
       -0.25350818,  0.48246992,  0.12052973,  0.08013797,  0.07505435,
        0.45640057, -0.6654021 , -0.44835666, -0.00786007, -0.5183773 ,
       -0.00484134, -0.01148336,  0.28142917, -0.01758763,  0.06659041,
        0.94368154, -0.16261122, -0.15630189,  0.30726194,  0.19

In [ ]:
### model.wv methods
#### 'most_similar()' and 'similarity()'

In [13]:
model.wv.most_similar('happiness')

[('despair', 0.7301496863365173),
 ('wherefore', 0.7294198274612427),
 ('virtue', 0.7269254922866821),
 ('jealousy', 0.7168462872505188),
 ('shakespearebr', 0.7144066095352173),
 ('arise', 0.7120428681373596),
 ('ignorance', 0.7045302391052246),
 ('shame', 0.703694224357605),
 ('kindness', 0.7031581401824951),
 ('psalm', 0.6967296004295349)]

In [14]:
model.wv.most_similar('furniture')

[('artwork', 0.6930415034294128),
 ('paints', 0.6927978992462158),
 ('chippendale', 0.6907036304473877),
 ('fastener', 0.6883490085601807),
 ('pottery', 0.6808674335479736),
 ('bicycles', 0.6789553165435791),
 ('ceramic', 0.6788221001625061),
 ('flooring', 0.6764292120933533),
 ('cabriole', 0.675460934638977),
 ('monogram', 0.6749094724655151)]

In [15]:
model.wv.similarity('furniture', 'jewelry')

0.6630453

In [16]:
model.wv.most_similar(positive=['cat', 'animal', 'pet', 'mammal'])

[('carnivore', 0.8128958344459534),
 ('cheetah', 0.7955524921417236),
 ('scavenger', 0.7922323346138),
 ('rodent', 0.7912538051605225),
 ('arachnid', 0.7891545295715332),
 ('parrot', 0.7882996797561646),
 ('shorthaired', 0.7842555642127991),
 ('wading', 0.7803277373313904),
 ('flightless', 0.7746297121047974),
 ('shorttailed', 0.7745615839958191)]

In [17]:
model.wv.most_similar(positive=['cat', 'animal'], negative='pet')

[('lizard', 0.3776397705078125),
 ('insect', 0.3743097484111786),
 ('breed', 0.36435168981552124),
 ('camel', 0.35821762681007385),
 ('marsupial', 0.35507845878601074),
 ('domesticated', 0.3544871509075165),
 ('rodent', 0.3539559841156006),
 ('extinction', 0.3437584340572357),
 ('monkey', 0.3437137007713318),
 ('sheep', 0.34092289209365845)]

In [18]:
model.wv.most_similar(positive=['king', 'woman'], negative='man', topn=3)

[('throne', 0.297690749168396),
 ('monarch', 0.27509433031082153),
 ('queen', 0.2577328681945801)]

In [19]:
model.wv.most_similar(positive='usa')

[('fargo', 0.6399749517440796),
 ('tyra', 0.617228090763092),
 ('npr', 0.6081739664077759),
 ('retitled', 0.5980387330055237),
 ('dogpatch', 0.5920142531394958),
 ('pageant', 0.5874358415603638),
 ('wichita', 0.5813845992088318),
 ('bombay', 0.5791819095611572),
 ('computergenerated', 0.5788818597793579),
 ('crossroads', 0.5769336223602295)]

In [20]:
model.wv.most_similar('canada')

[('zambia', 0.692581057548523),
 ('marianas', 0.6910848617553711),
 ('commonwealth', 0.6846738457679749),
 ('territories', 0.6586534976959229),
 ('uruguay', 0.6584171652793884),
 ('albania', 0.646291196346283),
 ('manitoba', 0.6458621621131897),
 ('yalta', 0.6444039940834045),
 ('chile', 0.6434510946273804),
 ('liechtenstein', 0.6419185996055603)]

In [21]:
model.wv.most_similar('shakespeare')

[('shakespeares', 0.7422709465026855),
 ('sophocles', 0.7291842103004456),
 ('euripides', 0.7068688869476318),
 ('romeo', 0.6914283037185669),
 ('ibsen', 0.6874734163284302),
 ('shakespearean', 0.6804429292678833),
 ('shaws', 0.6732524633407593),
 ('falstaff', 0.6704421043395996),
 ('hamlet', 0.6690429449081421),
 ('moliere', 0.6675474643707275)]

In [22]:
model.wv.most_similar(positive=['president', 'germany'], negative='usa')

[('emperors', 0.2684832811355591),
 ('dictator', 0.25363075733184814),
 ('ussr', 0.21678945422172546),
 ('regime', 0.2130197286605835),
 ('emperor', 0.19828079640865326),
 ('imprisoned', 0.1896682232618332),
 ('presidents', 0.18698829412460327),
 ('headlines', 0.18627069890499115),
 ('fascist', 0.18320727348327637),
 ('invaded', 0.18310856819152832)]

In [23]:
#### 'doesnt_match()'

In [24]:
model.wv.doesnt_match(['breakfast', 'lunch', 'frog', 'food'])

/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'frog'